In [ ]:
import sys

# Add the project root directory to Python path
sys.path.append('/Users/alexanderithakis/Documents/GitHub/D4PG-QR-FRM')

from run import generate_datasets
from src.swing_contract import SwingContract


# Create environments and contracts
swing_contract = SwingContract(
    q_min=0.0,
    q_max=2.0,
    Q_min=0.0,
    Q_max=20.0,
    strike=1.0,
    maturity=0.0833,
    n_rights=22,
    r=0.05,
    min_refraction_days=0,
)

stochastic_process_params = {
    "S0": 1.0,
    "T": 0.0833,
    "n_steps": 22,
    "alpha": 12.0,
    "sigma": 1.2,
    "beta": 150.0,
    "lam": 6.0,
    "mu_J": 0.3,
    "f": lambda t: 0.0,
}

# Generate training and evaluation datasets:
train_ds, eval_ds = generate_datasets(
    stochastic_process_params=stochastic_process_params,
    n_paths=args.n_paths,
    n_paths_eval=args.n_paths_eval,
    seed=seed,
)

In [2]:
# --- HHK/Kluge swing setup matching your simulator parameters ----------------
import math
import QuantLib as ql
import time

# Dates & conventions
todaysDate = ql.Date(30, ql.September, 2018)
ql.Settings.instance().evaluationDate = todaysDate
dc = ql.Actual365Fixed()

## Annual Contract with daily rights
# Parameters (your spec)
# S0    = 1.0
# K     = 1.0
# r     = 0.0
# T_yrs = 1.0
# n_rights = 365
# Q_min, Q_max = 0, 100

# alpha = 7.0       # OU speed
# sigma = 1.4       # OU vol
# beta  = 200.0     # jump decay
# lam   = 4.0       # jump intensity
# mu_J  = 0.4       # mean jump size
# eta   = 1.0/mu_J  # QL uses rate

## Monthly Contract with daily rights
# Monthly Swing Option Contract Parameters
S0    = 1.0
K     = 1.0
r     = 0.05
T_yrs = 0.0833
n_rights = 22
Q_min, Q_max = 0, 20

alpha = 12.0       # OU speed
sigma = 1.2       # OU vol
beta  = 150.0     # jump decay
lam   = 6.0       # jump intensity
mu_J  = 0.3       # mean jump size
eta   = 1.0/mu_J  # QL uses rate


# Market handles
riskFreeRate  = ql.FlatForward(todaysDate, r, dc)
dividendYield = ql.FlatForward(todaysDate, 0.0, dc)
underlying    = ql.SimpleQuote(S0)

# Exercise schedule: daily for 1 year
exerciseDates = [todaysDate + ql.Period(i, ql.Days) for i in range(1, n_rights + 1)]
exercise      = ql.SwingExercise(exerciseDates)

# Payoff: forward-style with strike K (bang-bang 0/1 per day)
payoff = ql.VanillaForwardPayoff(ql.Option.Call, K)

# Build the swing option with global volume limits
swing = ql.VanillaSwingOption(payoff, exercise, Q_min, Q_max)

# Deterministic log-shift f(t) (martingale shift; keeps E[S_t]=S0 when r=0)
def f(t: float) -> float:
    return ( math.log(S0)
             - (sigma**2)/(4.0*alpha) * (1.0 - math.exp(-2.0*alpha*t))
             - (lam/beta) * math.log((eta - math.exp(-beta*t)) / (eta - 1.0)) )

# Time grid for the shift
t_grid = [dc.yearFraction(todaysDate, d) for d in exerciseDates]
curveShape = list(zip(t_grid, [f(t) for t in t_grid]))

# Diffusive OU: X0 = log(S0) - f(0), long-run mean m(t)=0  → dX = -αX dt + σ dW
x0_X = math.log(S0) - f(0.0)   # this is 0 with the f(t) above; written explicitly for clarity
ouProcess = ql.ExtendedOrnsteinUhlenbeckProcess(
    alpha, sigma, x0_X, lambda tt: 0.0
)

# Jump OU: Y0 = 0, compound Poisson with Exp(rate=eta) marks
jProcess = ql.ExtOUWithJumpsProcess(ouProcess, 0.0, beta, lam, eta)

# FD engine (increase grids for accuracy; beta=200 is stiff in y)
# tGrid, xGrid, yGrid = 400, 200, 400 # for high accuracy - annual
def get_price_over_grid_size(grid_size_coef:float=1) -> float:
    _timer = time.perf_counter()
    tGrid, xGrid, yGrid = 25, 25, 50
    tGrid = int(tGrid * grid_size_coef)
    xGrid = int(xGrid * grid_size_coef)
    yGrid = int(yGrid * grid_size_coef)
    engine = ql.FdSimpleExtOUJumpSwingEngine(
        jProcess,
        riskFreeRate,            # pass the flat curve object itself
        tGrid, xGrid, yGrid,
        curveShape
    )

    swing.setPricingEngine(engine)
    price = swing.NPV()
    print(f"coef:{grid_size_coef} HHK/Kluge Swing NPV: {price} - execution time: {time.perf_counter() - _timer:.2f} sec")
    return price

coefs = [0.5, 0.75, 1]
for coef in coefs: get_price_over_grid_size(coef)

coef:0.5 HHK/Kluge Swing NPV: 1.4955886591008807 - execution time: 0.06 sec
coef:0.75 HHK/Kluge Swing NPV: 1.495157065184994 - execution time: 0.09 sec
coef:1 HHK/Kluge Swing NPV: 1.4954509165127619 - execution time: 0.19 sec


In [ ]:
import numpy as np

def simulate_extoujumps_paths(
    S0, alpha, sigma, beta, lam, eta, f, t_grid, n_paths=10000, seed=42
):
    """
    Returns S, X, Y with shape (n_paths, len(t_grid)+1).
    X follows dX = -alpha*X dt + sigma dW.
    Y follows dY = -beta*Y dt + dJ, where J is compound Poisson with Exp(rate=eta) marks.
    S_t = exp(f(t) + X_t + Y_t).
    """
    rng = np.random.default_rng(seed)
    n_steps = len(t_grid)

    X = np.zeros((n_paths, n_steps + 1))
    Y = np.zeros((n_paths, n_steps + 1))
    S = np.zeros((n_paths, n_steps + 1))
    S[:, 0] = np.exp(f(0.0) + X[:, 0] + Y[:, 0])  # equals S0 given your f(·)

    for k in range(n_steps):
        dt = t_grid[k] - (t_grid[k - 1] if k > 0 else 0.0)

        # --- exact OU step for X
        eadt = np.exp(-alpha * dt)
        var_X = (sigma**2) * (1.0 - np.exp(-2.0 * alpha * dt)) / (2.0 * alpha)
        X[:, k + 1] = eadt * X[:, k] + np.sqrt(var_X) * rng.standard_normal(n_paths)

        # --- jump-OU step for Y
        ebdt = np.exp(-beta * dt)
        Y[:, k + 1] = ebdt * Y[:, k]

        # Poisson number of jumps in (t, t+dt]
        N = rng.poisson(lam * dt, size=n_paths)
        has = N > 0
        if np.any(has):
            idx = np.where(has)[0]
            total = int(N[has].sum())

            # jump arrival times ~ Uniform(0, dt); marks ~ Exp(rate=eta)
            u = rng.random(total) * dt
            xi = rng.exponential(1.0 / eta, size=total)

            # assign jumps to paths; contribution is xi * exp(-beta*(dt - u))
            reps = np.repeat(idx, N[has])
            contrib = xi * np.exp(-beta * (dt - u))
            np.add.at(Y[:, k + 1], reps, contrib)

        # --- price
        tk = t_grid[k]
        S[:, k + 1] = np.exp(f(tk) + X[:, k + 1] + Y[:, k + 1])

    return S, X, Y


S_paths, X_paths, Y_paths = simulate_extoujumps_paths(
    S0, alpha, sigma, beta, lam, eta, f, t_grid, n_paths=20000, seed=1
)

tg = ql.TimeGrid([0.0] + t_grid)

# pick one simulated path and pack into a QuantLib Matrix (2 rows: X and Y)
p = 0
mat = ql.Matrix(2, tg.size())
for i in range(tg.size()):
    mat[0][i] = X_paths[p, i]
    mat[1][i] = Y_paths[p, i]
# Or build a ql.Path for S similarly, if that's what you consume later.


AttributeError: No constructor defined